In [1]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import ne_chunk
import nltk
from nltk.parse.corenlp import CoreNLPDependencyParser, CoreNLPParser

In [2]:
def pre_processing(textlines):
    if isinstance(textlines, str):
        tokens = word_tokenize(textlines)
        pos_tagger = nltk.pos_tag(tokens)
        ner_tags = ne_chunk(pos_tagger)
        return ner_tags
    else:
        sent_tokens = []
        for line in textlines:
            if len(line) > 0:
                sent_tokens = sent_tokens + sent_tokenize(line)

        word_tokens = []
        for sentence in sent_tokens:
            word_tokens.append(word_tokenize(sentence))

        pos_tagger = []
        for sent in word_tokens:
            pos_tagger.append(nltk.pos_tag(sent))

        ner_tags = []
        for sent in pos_tagger:
            ner_tags.append(ne_chunk(sent))
        return sent_tokens, word_tokens, ner_tags


def pos_tags_combining(pos_tagger):
    # Input: list of post tagged sentences
    for sent in pos_tagger:
        NNP_flag = False
        i = 0
        while i < len(sent):
            if sent[i][1] == 'NNP' and NNP_flag is False:
                NNP_flag = True
            elif sent[i][1] == 'NNP' and NNP_flag is True:
                word = sent[i - 1][0] + ' ' + sent[i][0]
                sent[i - 1] = (word, 'NNP')
                sent.remove(sent[i])
            elif sent[i][1] != 'NNP' and NNP_flag is True:
                NNP_flag = False
            i = i + 1

In [3]:
fd = open('a2.txt', encoding="utf8")
text = fd.read()
textlines = text.split('\n')

sent_tokens, word_tokens, ner_tags = pre_processing(textlines)

parses = []
parser = CoreNLPParser(url='http://localhost:9000')
# parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [39]:
# parsed_sentences = parser.raw_parse_sents(sent_tokens[:1])
# parsed_sentences = [next(i) for i in parsed_sentences]

In [40]:
# print(parsed_sentences[0])
# parsed_sentences[0].pretty_print()

In [41]:
# sent = "Ash is running away"
# itr = parser.raw_parse_sents([sent])
# print(itr)

In [42]:
# trees = [next(i) for i in itr]
# print(trees)

In [43]:
# trees[0]

In [44]:
# sent = "Ash is running away"
# tagged_sent = nltk.pos_tag(sent.split())
# tagged_sent

In [64]:
def who_quest(pos_tagged_sent):
    noun_tags = set(['NNP', 'NNPS'])
    verb_tags = set(['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'MD'])
    punctuations = set(['.', ',', '(', ')', '[', ']', ';'])
#     # Find the number of NNPs
#     all_NNP = set()
#     for w in pos_tagged_sent:
#         if w[1]=='NNP':
#             all_NNP.add(w)
    # Find those NNP which are follwoed by verbs
    verb_NNP = set()
    for i,w in enumerate(pos_tagged_sent):
        if i<len(pos_tagged_sent)-1 and w[1]=='NNP' and pos_tagged_sent[i+1][1] in verb_tags:
            verb_NNP.add(w)
    # Make questions for all the NNPs one by one
#     questions = []
#     for nnp in all_NNP:
#         quest = []
#         for w in pos_tagged_sent:
#             if w==nnp:
#                 quest += ['who']
#             else:
#                 quest += [w[0]]
#         questions.append(' '.join(quest)+'?')
    # Make question if noun followed by verb
    questions = []
    start = 0
    for nnp in verb_NNP:
        quest = []
        for w in pos_tagged_sent:
            if w==nnp:
                quest += ['who']
                start = 1
            else:
                if start:
                    if w[1] in punctuations:
                        break
                    quest += [w[0]]
                    
        questions.append(' '.join(quest)+'?')
    return questions

In [66]:
sentences = sent_tokens[0:100]
# sentences = ["Arindam ran away from the Professor"]
# print(sentences)
tokenized_sents = [word_tokenize(sent) for sent in sentences]
# print(tokenized_sents)
pos_tagged_sents = [nltk.pos_tag(sent) for sent in tokenized_sents]
pos_tags_combining(pos_tagged_sents)
# print(pos_tagged_sents[11])
# print(who_quest(pos_tagged_sents[11]))

In [67]:
questions = [who_quest(sent) for sent in pos_tagged_sents]
# print(questions)
for quest in questions:
    for q in quest:
        print(q)

who has appeared in all episodes of the anime?
who has gone on to become one of the most well-known and recognizable animated characters of all-time?
who leagues as of yet?
who was as a child?
who revived?
who noted in an interview that between Japanese and American reactions to the series?
who legend Shigeru Miyamoto?
who noted the contrast between the characters ' relationship in the games and anime ; while in the games they were rivals?
In an interview Tajiri noted the contrast between the characters ' relationship in the games and anime ; while in the games they were rivals?
who replied `` No?
When asked if either who would equal or surpass Shigeru?
who storyline arc?
who received a redesign in the Best Wishes?
who received a major design overhaul for the Sun and Moon anime series?
who has always provided the original Japanese voice of Ash?
who provided the voice of Ash in the first eight seasons of the English adaption of the Pokémon anime?
who stated that the character was tempor